In [ ]:
import alpaca_trade_api as tradeapi
api_key='YOUR API KEY'
api_secret='YOUR API SECRET'
my_url='https://paper-api.alpaca.markets/'
api = tradeapi.REST(api_key, api_secret, base_url=my_url)
account = api.get_account()
print(account.status)  # Should print 'ACTIVE'

ACTIVE


In [2]:
# Backtesting

import pandas as pd
from alpaca_trade_api.rest import REST, TimeFrame
import alpaca_trade_api as tradeapi
from datetime import timedelta
from tqdm import tqdm

# Parameters for backtesting
symbol = 'AAPL'
momentum_threshold = 0.01  # 1% price change threshold
cooldown_period = timedelta(minutes=5)  # Cooldown period
trading_fee = 0.001  # 0.1% trading fee per transaction
initial_cash = 100000  # Starting cash

# Fetch historical data from Alpaca
start_date = '2023-01-01'
end_date = '2023-12-01'
bars = api.get_bars(symbol, TimeFrame.Minute, start=start_date, end=end_date).df

# Ensure the DataFrame has valid timestamps
bars = bars[['close']]
bars.index = pd.to_datetime(bars.index)

# Parameters adjustments
volatility_window = 15  # Lookback period for volatility
risk_reward_ratio = 2  # Minimum risk-reward ratio
take_profit = 0.03  # 3% profit to close position
stop_loss = 0.01  # 1% loss to close position

bars['volatility'] = bars['close'].rolling(volatility_window).std()

# Precompute rolling statistics
bars['mean_price'] = bars['close'].rolling(15).mean()
bars['volatility'] = bars['close'].rolling(15).std()

def backtest_with_optimized_loops(data, initial_cash, trading_fee, risk_reward_ratio, take_profit, stop_loss):
    cash = initial_cash
    shares = 0
    last_price = None
    trade_log = []
    last_trade_time = None

    for current_time, row in tqdm(data.iterrows(), total=len(data)):
        current_close = row['close']
        current_mean = row['mean_price']
        current_volatility = row['volatility']

        if pd.isna(current_volatility) or last_price is None:
            last_price = current_close
            continue

        price_change = (current_close - last_price) / last_price

        # Enforce minimum holding period
        if last_trade_time and shares > 0 and (current_time - last_trade_time) < timedelta(minutes=10):
            continue

        # Stop loss and take profit
        if shares > 0:
            potential_profit = (current_close - last_price) / last_price
            if potential_profit >= take_profit or potential_profit <= -stop_loss:
                cash += shares * current_close * (1 - trading_fee)
                trade_log.append({'action': 'sell', 'time': current_time, 'price': current_close, 'shares': shares})
                shares = 0
                last_trade_time = current_time
                last_price = current_close
                continue

        # Additional sell signal: Strong Momentum Reversal
        if shares > 0 and price_change < -2 * (0.02 * current_volatility):
            cash += shares * current_close * (1 - trading_fee)
            trade_log.append({'action': 'sell', 'time': current_time, 'price': current_close, 'shares': shares})
            shares = 0
            last_trade_time = current_time
            last_price = current_close
            continue

        # Buy signal with RRR
        if price_change > 0.02 * current_volatility:
            potential_risk = stop_loss * last_price
            potential_reward = take_profit * last_price
            if potential_reward / potential_risk >= risk_reward_ratio:
                shares_to_buy = (cash * 0.5) // current_close
                if shares_to_buy > 0:
                    cash -= shares_to_buy * current_close * (1 + trading_fee)
                    shares += shares_to_buy
                    trade_log.append({'action': 'buy', 'time': current_time, 'price': current_close, 'shares': shares_to_buy})
                    last_trade_time = current_time

        last_price = current_close

    # Final portfolio value
    final_portfolio_value = cash + (shares * data['close'].iloc[-1])
    return {
        'final_portfolio_value': final_portfolio_value,
        'cash': cash,
        'shares': shares,
        'trade_log': pd.DataFrame(trade_log),
        'total_trades': len(trade_log),
    }

# Run the backtest
results = backtest_with_optimized_loops(
    bars, initial_cash, trading_fee, risk_reward_ratio, take_profit, stop_loss
)

# Display results
print("Final Portfolio Value:", results['final_portfolio_value'])
print("Cash Remaining:", results['cash'])
print("Shares Held:", results['shares'])
print("Total Trades:", results['total_trades'])
print("Trade Log:")
print(results['trade_log'])


100%|██████████| 173823/173823 [00:11<00:00, 14868.06it/s]

Final Portfolio Value: 145726.13462410014
Cash Remaining: 315.3346241001337
Shares Held: 760.0
Total Trades: 126
Trade Log:
    action                      time     price  shares
0      buy 2023-01-03 18:25:00+00:00  124.5700   401.0
1      buy 2023-01-03 23:01:00+00:00  124.9000   200.0
2      buy 2023-01-04 00:21:00+00:00  125.0500    99.0
3      buy 2023-01-04 00:55:00+00:00  125.0500    50.0
4      buy 2023-01-04 09:00:00+00:00  126.3400    25.0
..     ...                       ...       ...     ...
121    buy 2023-11-08 21:47:00+00:00  182.8900    11.0
122    buy 2023-11-09 09:00:00+00:00  182.6900     6.0
123    buy 2023-11-13 14:27:00+00:00  185.7400     3.0
124    buy 2023-11-14 13:30:00+00:00  186.4215     1.0
125    buy 2023-11-15 09:02:00+00:00  187.8200     1.0

[126 rows x 4 columns]


In [3]:
# Real-time Trading

import time
from datetime import datetime, timedelta
import pandas as pd
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame
import pytz  # For timezone handling
import logging  # For logging

# Alpaca API Configuration
API_KEY = 'PKAYUGRGOEZBCUCX6EYC'       
API_SECRET = '3F29OtqbTQEKCZeVmSSIfLaYWmiWEbQsDVPcsX8J' 
BASE_URL_TRADING = 'https://paper-api.alpaca.markets'  # Use this URL for paper trading

# Initialize Alpaca API (Trading API only)
api_trading = REST(API_KEY, API_SECRET, BASE_URL_TRADING, api_version='v2')

# Trading Parameters
symbol = 'AAPL'  # Stock symbol
momentum_threshold = 0.01  # 1% price change threshold
cooldown_period = timedelta(minutes=5)  # Trading interval cooldown period
trading_fee = 0.001  # 0.1% fee per trade
initial_cash = 100000  # Initial capital

# Strategy Parameters
volatility_window = 15  # Lookback period for volatility calculation (minutes)
risk_reward_ratio = 2  # Minimum risk-reward ratio
take_profit = 0.03  # 3% profit target
stop_loss = 0.01  # 1% stop loss

# Initialize Account Status
cash = initial_cash
shares = 0
last_price = None
last_trade_time = None

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Configure logging to file and console
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Check if handlers are already added to avoid duplication
if not logger.handlers:
    # File handler
    file_handler = logging.FileHandler('trading_bot.log')
    file_handler.setLevel(logging.INFO)
    file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    console_handler.setFormatter(console_formatter)
    logger.addHandler(console_handler)

# Function to calculate price change
def calculate_change(current, previous):
    if previous == 0:
        return 0
    return (current - previous) / previous

# Safely fetch the latest price
def fetch_latest_price(symbol):
    try:
        trade = api_trading.get_latest_trade(symbol)  # Get the latest trade data
        if trade:
            current_price = trade.price
            current_time = datetime.now(eastern)
            return current_price, current_time
        else:
            logging.warning(f"No latest trade data for {symbol}.")
            return None, None
    except Exception as e:
        logging.error(f"Error fetching latest price: {e}")
        return None, None

# Function to check if the market is open
def is_market_open():
    try:
        clock = api_trading.get_clock()
        return clock.is_open
    except Exception as e:
        logging.error(f"Error fetching market clock: {e}")
        return False  # Assume market is closed if clock cannot be fetched

# Function to get current position
def get_current_position(symbol):
    try:
        position = api_trading.get_position(symbol)
        return int(position.qty)
    except tradeapi.rest.APIError as e:
        if e.status_code == 404:
            return 0  # No position
        else:
            logging.error(f"Error fetching position: {e}")
            return 0

# Function to get the last trading day
def get_last_trading_day():
    today = datetime.now(eastern).date()
    delta = timedelta(days=1)
    while True:
        last_day = today - delta  # Move back one day
        try:
            calendar = api_trading.get_calendar(start=last_day, end=last_day)
            if len(calendar) > 0:
                return last_day
        except Exception as e:
            logging.error(f"Error fetching trading calendar: {e}")
            return None
        today = last_day

# Initialize price history as empty (do not fetch previous day's data)
price_history = []
shares = get_current_position(symbol)
logging.info(f"Initial number of shares held: {shares}")

# Main trading loop
while True:
    try:
        # Check if the market is open
        market_open = is_market_open()
        if not market_open:
            logging.info("Market is closed. Waiting for the market to open...")
            time.sleep(60)  # Check every minute
            continue

        # Get the latest trade price
        current_price, current_time = fetch_latest_price(symbol)
        if current_price is None or current_time is None:
            time.sleep(60)
            continue

        # Update price history
        price_history.append(current_price)
        if len(price_history) > volatility_window:
            price_history.pop(0)

        # Ensure there is enough data to calculate volatility
        if len(price_history) < volatility_window:
            logging.info(f"Accumulating data... ({len(price_history)}/{volatility_window}) Current price: {current_price}")
            last_price = current_price
            time.sleep(60)
            continue

        # Calculate volatility and dynamic momentum threshold
        volatility = pd.Series(price_history).std()
        dynamic_momentum_threshold = 0.02 * volatility  # Adjust based on backtesting logic

        # Calculate price change
        if last_price is not None:
            price_change = calculate_change(current_price, last_price)
        else:
            price_change = 0

        logging.info(f"Current price: {current_price:.2f}, Volatility: {volatility:.4f}, Price change: {price_change:.4f}")

        # Check cooldown period
        if last_trade_time and (current_time - last_trade_time) < cooldown_period:
            in_cooldown = True
        else:
            in_cooldown = False

        # Check take profit and stop loss
        if shares > 0:
            potential_profit = (current_price - last_price) / last_price
            logging.info(f"Potential profit: {potential_profit:.4f}")
            if potential_profit >= take_profit or potential_profit <= -stop_loss:
                try:
                    # Execute sell order
                    response = api_trading.submit_order(
                        symbol=symbol,
                        qty=shares,
                        side='sell',
                        type='market',
                        time_in_force='gtc'
                    )
                    logging.info(f"Submitted sell order: {response}")
                    cash += shares * current_price * (1 - trading_fee)
                    logging.info(f"Sold {shares} shares at {current_price:.2f}. New cash balance: {cash:.2f}")
                    shares = 0
                    last_trade_time = current_time
                    last_price = current_price
                    time.sleep(60)
                    continue
                except Exception as e:
                    logging.error(f"Error executing sell order: {e}")

        # Additional sell signal: strong momentum reversal
        if shares > 0 and price_change < -2 * dynamic_momentum_threshold:
            try:
                # Execute sell order
                response = api_trading.submit_order(
                    symbol=symbol,
                    qty=shares,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )
                logging.info(f"Submitted strong momentum reversal sell order: {response}")
                cash += shares * current_price * (1 - trading_fee)
                logging.info(f"Sold {shares} shares at {current_price:.2f}. New cash balance: {cash:.2f}")
                shares = 0
                last_trade_time = current_time
                last_price = current_price
                time.sleep(60)
                continue
            except Exception as e:
                logging.error(f"Error executing strong momentum reversal sell order: {e}")

        # Buy signal: meets risk-reward ratio
        if price_change > dynamic_momentum_threshold and not in_cooldown:
            # Calculate potential risk and reward
            potential_risk = stop_loss * last_price if last_price else stop_loss * current_price
            potential_reward = take_profit * last_price if last_price else take_profit * current_price
            if potential_reward / potential_risk >= risk_reward_ratio:
                # Calculate number of shares to buy (using 50% of available cash)
                available_cash = cash * 0.5
                shares_to_buy = int((available_cash * (1 - trading_fee)) // current_price)
                if shares_to_buy > 0:
                    try:
                        # Submit buy order
                        response = api_trading.submit_order(
                            symbol=symbol,
                            qty=shares_to_buy,
                            side='buy',
                            type='market',
                            time_in_force='gtc'
                        )
                        logging.info(f"Submitted buy order: {response}")
                        # Update account status
                        cash -= shares_to_buy * current_price * (1 + trading_fee)
                        shares += shares_to_buy
                        last_trade_time = current_time
                        logging.info(f"Bought {shares_to_buy} shares at {current_price:.2f}. New cash balance: {cash:.2f}, Total shares held: {shares}")
                    except Exception as e:
                        logging.error(f"Error executing buy order: {e}")

        # Update last price
        last_price = current_price

        # Wait for the next minute
        time.sleep(60)

    except Exception as e:
        logging.error(f"Error in main loop: {e}")
        time.sleep(60)  # Wait a minute before retrying after an error


2024-12-04 13:22:56 - INFO - Initial number of shares held: 0
2024-12-04 13:22:56 - INFO - Accumulating data... (1/15) Current price: 242.975
2024-12-04 13:23:56 - INFO - Accumulating data... (2/15) Current price: 243.13
2024-12-04 13:24:56 - INFO - Accumulating data... (3/15) Current price: 243.1
2024-12-04 13:25:56 - INFO - Accumulating data... (4/15) Current price: 243.1
2024-12-04 13:26:56 - INFO - Accumulating data... (5/15) Current price: 243.14
2024-12-04 13:27:56 - INFO - Accumulating data... (6/15) Current price: 243.07
2024-12-04 13:28:56 - INFO - Accumulating data... (7/15) Current price: 243.01
2024-12-04 13:29:56 - INFO - Accumulating data... (8/15) Current price: 243.065
2024-12-04 13:30:56 - INFO - Accumulating data... (9/15) Current price: 242.96
2024-12-04 13:31:57 - INFO - Accumulating data... (10/15) Current price: 243.015
2024-12-04 13:32:57 - INFO - Accumulating data... (11/15) Current price: 243.03
2024-12-04 13:33:57 - INFO - Accumulating data... (12/15) Current 

KeyboardInterrupt: 